In [14]:
import torch, math, time, argparse, os
import random, dataset, utils, losses, net
import numpy as np

from dataset.market import Market
from net.os_net import get_embedding_model, osnet_ibn_x1_0, OSNet, OSBlock
from dataset import sampler
from torch.utils.data.sampler import BatchSampler
from torch.utils.data.dataloader import default_collate
from torch import nn


from tqdm import *
import wandb

In [2]:
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [3]:
LOG_DIR = '.'
trn_dataset = dataset.load('market', '../ITCS-5145-CV/learning/Market-1501-v15.09.15/', 'train', transform = dataset.utils.make_transform(
                                                                                                                        is_train = True, 
                                                                                                                        is_inception = False))

In [4]:
dl_tr = torch.utils.data.DataLoader(
        trn_dataset,
        batch_size = 50,
        shuffle = True,
        num_workers = 4,
        drop_last = True,
        pin_memory = True
    )

In [5]:
eval_dataset = dataset.load('market', '../ITCS-5145-CV/learning/Market-1501-v15.09.15/', 'train', transform = dataset.utils.make_transform(
                                                                                                                        is_train = True, 
                                                                                                                        is_inception = False))

dl_ev = torch.utils.data.DataLoader(
        eval_dataset,
        batch_size = 50,
        shuffle = False,
        num_workers = 4,
        pin_memory = True
)

In [6]:
model_name = 'OSNET'

In [7]:
model = osnet_ibn_x1_0(pretrained=True).cuda()

Successfully loaded imagenet pretrained weights from "/home/nevin/.cache/torch/checkpoints/osnet_ibn_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [8]:
criterion = losses.Proxy_Anchor(nb_classes = trn_dataset.nb_classes(), sz_embed = 512, mrg = 0.1, alpha = 32).cuda()

In [9]:
param_groups = [
    {'params': list(set(model.parameters()).difference(set(model.fc.parameters())))},
    {'params': model.fc.parameters(), 'lr':float(1e-4) * 1},
]
param_groups.append({'params': criterion.parameters(), 'lr':float(1e-4) * 100})

In [10]:
opt = torch.optim.AdamW(param_groups, lr=float(1e-4), weight_decay = 1e-4)

In [11]:
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=10, gamma = 0.5)

In [12]:
model_dir = 'models'

In [16]:
best_recall=[0]
log = {}
for epoch in range(0, 60):
    model.train()
    bn_freeze = True

    if bn_freeze:
            modules = model.modules()
            for m in modules: 
                if isinstance(m, nn.BatchNorm2d):
                    m.eval()

    losses_per_epoch = []
    unfreeze_model_param = list(model.fc.parameters()) + list(criterion.parameters())

    if epoch == 0:
        for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
            param.requires_grad = False
    if epoch == 1:
        for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
            param.requires_grad = True

    pbar = tqdm(enumerate(dl_tr))

    for batch_idx, (x, y) in pbar:                         
        m = model(x.squeeze().cuda())
        loss = criterion(m, y.squeeze().cuda())
        
        opt.zero_grad()
        loss.backward()
        
        torch.nn.utils.clip_grad_value_(model.parameters(), 10)
        
        torch.nn.utils.clip_grad_value_(criterion.parameters(), 10)

        losses_per_epoch.append(loss.data.cpu().numpy())
        opt.step()

        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f}'.format(
                epoch, batch_idx + 1, len(dl_tr),
                100. * batch_idx / len(dl_tr),
                loss.item()))
    scheduler.step()
    
    if(epoch >= 0):
        with torch.no_grad():
            print("**Evaluating...**")
            Recalls = utils.evaluate_cos(model, dl_ev)
        if best_recall[0] < Recalls[0]:
            best_recall = Recalls
            best_epoch = epoch
            if not os.path.exists(model_dir):
                os.makedirs(model_dir)
            torch.save({'model_state_dict':model.state_dict()}, '{}/{}_best.pth'.format(model_dir, model_name))
            with open('{}/{}_best_results.txt'.format(model_dir, model_name), 'w') as f:
                f.write('Best Epoch: {}\n'.format(best_epoch))
                for i in range(6):
                    f.write("Best Recall@{}: {:.4f}\n".format(2**i, best_recall[i] * 100))

0it [00:00, ?it/s]/home/nevin/anaconda3/envs/cv/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
Train Epoch: 0 [549/549 (100%)] Loss: 2.588382: : 549it [00:51, 10.69it/s]


**Evaluating...**


100%|█████████████████████████████████████████| 550/550 [00:48<00:00, 11.28it/s]


R@1 : 28.088
R@2 : 34.129
R@4 : 40.857
R@8 : 47.844
R@16 : 55.329
R@32 : 63.804


Train Epoch: 1 [549/549 (100%)] Loss: 1.709753: : 549it [02:32,  3.60it/s]


...

R@1 : 74.979
R@2 : 80.619
R@4 : 85.270
R@8 : 88.996
R@16 : 92.082
R@32 : 94.483


Train Epoch: 59 [549/549 (100%)] Loss: 0.387693: : 549it [02:27,  3.73it/s]


**Evaluating...**


100%|█████████████████████████████████████████| 550/550 [00:49<00:00, 11.10it/s]


R@1 : 74.630
R@2 : 80.310
R@4 : 85.084
R@8 : 88.876
R@16 : 92.169
R@32 : 94.720
